In [2]:
!pip install requests beautifulsoup4 geopy pytz geocoder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   -----------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\piyus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import requests
from geopy.geocoders import Nominatim
import datetime
import pytz
import time
import geocoder

Geolocator instance with proper user_agent

In [4]:
geolocator = Nominatim(user_agent="krishi_sowing_predictor_2025")

Map lat-long to Indian state using geopy

In [5]:
def get_state_from_latlong(lat, lon):
    try:
        location = geolocator.reverse(f"{lat}, {lon}", language='en', addressdetails=True, timeout=10)
        time.sleep(1)
        if location and 'state' in location.raw['address']:
            return location.raw['address']['state']
    except Exception as e:
        print("Geocoding error:", e)
    return None

Real-time weather scraping from Google

In [6]:
def fetch_wttr_weather(city):
    url = f"https://wttr.in/{city}?format=j1"
    try:
        resp = requests.get(url, timeout=5)
        data = resp.json()
        temp_c = float(data['current_condition'][0]['temp_C'])
        condition = data['current_condition'][0]['weatherDesc'][0]['value']
        return temp_c, condition
    except Exception as e:
        print("Error fetching wttr.in weather:", e)
        return None, None

Rule-based sowing calendar for major Indian states and crops (approximate)

In [7]:
SOWING_CALENDAR = {
    "Andhra Pradesh": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Arunachal Pradesh": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Assam": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Bihar": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Chhattisgarh": {
        "Wheat": (10, 11),
        "Sugarcane": (1, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Goa": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Gujarat": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Haryana": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Himachal Pradesh": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Jharkhand": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Karnataka": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Kerala": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Madhya Pradesh": {
        "Wheat": (10, 11),
        "Sugarcane": (1, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Maharashtra": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Manipur": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Meghalaya": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Mizoram": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Nagaland": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Odisha": {
        "Wheat": (10, 11),
        "Sugarcane": (1, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Punjab": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Rajasthan": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Sikkim": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Tamil Nadu": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Telangana": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Tripura": {
        "Wheat": (11, 12),
        "Sugarcane": (2, 4),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Uttar Pradesh": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Uttarakhand": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "West Bengal": {
        "Wheat": (10, 11),
        "Sugarcane": (1, 3),
        "Potato": (10, 12),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    # Union Territories
    "Andaman and Nicobar Islands": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Chandigarh": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Dadra and Nagar Haveli and Daman and Diu": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Delhi": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Jammu and Kashmir": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Ladakh": {
        "Wheat": (10, 11),
        "Sugarcane": (2, 3),
        "Potato": (9, 11),
        "Corn": (6, 7),
        "Rice": (6, 7),
    },
    "Lakshadweep": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
    "Puducherry": {
        "Wheat": (11, 12),
        "Sugarcane": (1, 3),
        "Potato": (10, 11),
        "Corn": (6, 8),
        "Rice": (6, 8),
    },
}

In [8]:
CROP_VARIETY_RECOMMENDATIONS = {
    "Andhra Pradesh": {
        "Wheat": [
            {"name": "Sona", "traits": ["Heat tolerant", "Good yield"]},
            {"name": "MACS 6222", "traits": ["Disease resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content", "Pest resistant"]},
            {"name": "Co 94012", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
            {"name": "Kufri Badshah", "traits": ["Good for storage"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
            {"name": "DHM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "MTU 1010", "traits": ["High yield", "Disease resistant"]},
            {"name": "BPT 5204", "traits": ["Good grain quality"]},
        ],
    },
    "Arunachal Pradesh": {
        "Wheat": [
            {"name": "NHL 15", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Nutrient rich", "Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
            {"name": "Kalinath", "traits": ["Traditional variety"]},
        ],
    },
    "Assam": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["High yield", "Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content", "Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
        ],
        "Rice": [
            {"name": "Jalashree", "traits": ["Flood tolerant"]},
            {"name": "Madhukar", "traits": ["High yield"]},
        ],
    },
    "Bihar": {
        "Wheat": [
            {"name": "Kalyan Sona", "traits": ["Heat tolerant", "High yield"]},
            {"name": "HD 2967", "traits": ["Rust resistant", "Drought tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant", "Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Chandramukhi", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
        ],
        "Rice": [
            {"name": "Swarna", "traits": ["Drought tolerant"]},
            {"name": "PR 126", "traits": ["Bacterial leaf blight resistant"]},
        ],
    },
    "Chhattisgarh": {
        "Wheat": [
            {"name": "HD 2329", "traits": ["High yield"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Bahar", "traits": ["Heat tolerant"]},
        ],
        "Corn": [
            {"name": "Ganga 11", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Moti", "traits": ["High yield"]},
        ],
    },
    "Goa": {
        "Wheat": [
            {"name": "Sonalika", "traits": ["Good quality"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Pukhraj", "traits": ["Drought tolerant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "MTU 1010", "traits": ["Good yield"]},
        ],
    },
    "Gujarat": {
        "Wheat": [
            {"name": "GW 273", "traits": ["Heat tolerant", "Good yield"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content", "Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Gujarat Hybrid Corn 2", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "GNR 1", "traits": ["Salt tolerant"]},
        ],
    },
    "Haryana": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["Rust resistant", "High yield"]},
            {"name": "PBW 550", "traits": ["Drought tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Disease resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "PR 126", "traits": ["High yield"]},
        ],
    },
    "Himachal Pradesh": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Jharkhand": {
        "Wheat": [
            {"name": "HD 2733", "traits": ["Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "Swarna", "traits": ["Drought tolerant"]},
        ],
    },
    "Karnataka": {
        "Wheat": [
            {"name": "MACS 6222", "traits": ["Disease resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "DHPM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "MTU 1001", "traits": ["High yield"]},
        ],
    },
    "Kerala": {
        "Wheat": [
            {"name": "MACS 6222", "traits": ["Disease resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "DHPM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Jyothi", "traits": ["High yield"]},
        ],
    },
    "Madhya Pradesh": {
        "Wheat": [
            {"name": "GW 273", "traits": ["Heat tolerant"]},
            {"name": "HI 8498", "traits": ["Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Bahar", "traits": ["Heat tolerant"]},
        ],
        "Corn": [
            {"name": "Ganga 11", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Jaya", "traits": ["High yield"]},
        ],
    },
    "Maharashtra": {
        "Wheat": [
            {"name": "GW 273", "traits": ["Heat tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Bahar", "traits": ["Heat tolerant"]},
        ],
        "Corn": [
            {"name": "Nashik 9", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "MTU 1001", "traits": ["Good yield"]},
        ],
    },
    "Manipur": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Meghalaya": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Mizoram": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Nagaland": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Odisha": {
        "Wheat": [
            {"name": "HD 2733", "traits": ["Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Ganga 11", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Swarna", "traits": ["Drought tolerant"]},
            {"name": "Lalat", "traits": ["High yield"]},
        ],
    },
    "Punjab": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["High yield", "Rust resistant"]},
            {"name": "PBW 550", "traits": ["Drought tolerant", "Good quality"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant", "Early maturity"]},
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
            {"name": "Kufri Pukhraj", "traits": ["Drought tolerant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
            {"name": "HQPM 5", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "PR 126", "traits": ["Bacterial leaf blight resistant"]},
            {"name": "Swarna", "traits": ["Drought tolerant"]},
        ],
    },
    "Rajasthan": {
        "Wheat": [
            {"name": "Raj 3765", "traits": ["Heat tolerant"]},
            {"name": "Raj 4079", "traits": ["Drought tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Lalima", "traits": ["Heat tolerant"]},
        ],
        "Corn": [
            {"name": "Ganga 11", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Lalat", "traits": ["Heat tolerant"]},
        ],
    },
    "Sikkim": {
        "Wheat": [
            {"name": "NHL 15", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Tamil Nadu": {
        "Wheat": [
            {"name": "MACS 6222", "traits": ["Disease resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
            {"name": "Co 94012", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "DHPM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "ADT 43", "traits": ["High yield"]},
        ],
    },
    "Telangana": {
        "Wheat": [
            {"name": "Sona", "traits": ["Heat tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "DHPM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "MTU 1010", "traits": ["High yield"]},
        ],
    },
    "Tripura": {
        "Wheat": [
            {"name": "NHL 15", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "Uttar Pradesh": {
        "Wheat": [
            {"name": "Kalyan Sona", "traits": ["Heat tolerant", "High yield"]},
            {"name": "HD 2967", "traits": ["Rust resistant", "Drought tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
            {"name": "Co 0238", "traits": ["Pest resistant", "Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Chandramukhi", "traits": ["Late blight resistant"]},
            {"name": "Kufri Jyoti", "traits": ["Drought tolerant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
            {"name": "HQPM 5", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Swarna", "traits": ["Drought tolerant"]},
            {"name": "PR 126", "traits": ["Bacterial leaf blight resistant"]},
        ],
    },
    "Uttarakhand": {
        "Wheat": [
            {"name": "NHL 17", "traits": ["Cold tolerant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Pest resistant"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "Vivek QPM 9", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "Dular", "traits": ["Cold tolerant"]},
        ],
    },
    "West Bengal": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["High yield", "Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 86032", "traits": ["High sugar content"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield", "Pest resistant"]},
        ],
        "Rice": [
            {"name": "Swarna", "traits": ["Drought tolerant"]},
            {"name": "Lalat", "traits": ["High yield"]},
        ],
    },
    # Union Territories (basic recommendations)
    "Delhi": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "PR 126", "traits": ["Bacterial leaf blight resistant"]},
        ],
    },
    "Chandigarh": {
        "Wheat": [
            {"name": "PBW 343", "traits": ["Rust resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 0238", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "HQPM 1", "traits": ["High yield"]},
        ],
        "Rice": [
            {"name": "PR 126", "traits": ["Bacterial leaf blight resistant"]},
        ],
    },
    "Puducherry": {
        "Wheat": [
            {"name": "MACS 6222", "traits": ["Disease resistant"]},
        ],
        "Sugarcane": [
            {"name": "Co 94012", "traits": ["Early maturity"]},
        ],
        "Potato": [
            {"name": "Kufri Jyoti", "traits": ["Late blight resistant"]},
        ],
        "Corn": [
            {"name": "DHPM 117", "traits": ["Drought tolerant"]},
        ],
        "Rice": [
            {"name": "ADT 43", "traits": ["High yield"]},
        ],
    },
    # Add more UTs as needed with similar pattern...
}

In [43]:
crop_conditions = {
        "Rice": {
            "temp_range": (20, 35),
            "avoid": ["drought", "frost"]
        },
        "Wheat": {
            "temp_range": (10, 25),
            "avoid": ["heavy rain", "flood"]
        },
        "Sugarcane": {
            "temp_range": (20, 38),
            "avoid": ["frost", "drought"]
        },
        "Potato": {
            "temp_range": (15, 25),
            "avoid": ["frost", "waterlogging"]
        }
    }

Logic to decide sowing based on date, weather, crop

In [44]:
def is_sowing_time(state, crop, current_month, temperature, condition):
    # Normalize keys
    state = state.strip().title()
    crop = crop.strip().title()

    if state not in SOWING_CALENDAR:
        print(f"No calendar info for state '{state}', assuming no sowing window.")
        return False
    if crop not in SOWING_CALENDAR[state]:
        print(f"No calendar info for crop '{crop}' in state '{state}'.")
        return False

    start, end = SOWING_CALENDAR[state][crop]
    if start <= end:
        in_window = start <= current_month <= end
    else:
        in_window = current_month >= start or current_month <= end

    if crop not in crop_conditions:
        print(f"No specific conditions defined for crop '{crop}', using defaults.")
        suitable_temp = 15 <= temperature <= 35 if temperature is not None else False
        unsuitable_conditions = ["rain", "thunderstorm", "snow"]
    else:
        cond = crop_conditions[crop]
        tmin, tmax = cond["temp_range"]
        suitable_temp = tmin <= temperature <= tmax if temperature is not None else False
        unsuitable_conditions = cond["avoid"]

    suitable_condition = all(
        bad_cond.lower() not in condition.lower()
        for bad_cond in unsuitable_conditions
    ) if condition else False

    return in_window and suitable_temp and suitable_condition


In [45]:
def get_variety_recommendations(state, crop):
    # Normalize keys
    state = state.strip().title()
    crop = crop.strip().title()

    varieties = CROP_VARIETY_RECOMMENDATIONS.get(state, {}).get(crop, [])
    if not varieties:
        return "No specific variety recommendations available."
    
    recs = []
    for v in varieties:
        traits_str = ", ".join(v["traits"])
        recs.append(f"- {v['name']} ({traits_str})")
    return "\n".join(recs)


In [46]:
def get_current_location():
    """
    Fetches current latitude and longitude based on IP address.
    (For more accuracy, GPS hardware or browser permissions are needed.)
    """
    g = geocoder.ip('me')
    if g.ok:
        lat, lng = g.latlng
        return lat, lng
    else:
        raise Exception("Unable to fetch location")

In [ ]:
def predict_sowing_time(crop):
    # Detect location
    lat, lon = get_current_location()
    print(f"Detected Location: Latitude={lat}, Longitude={lon}")
    state = get_state_from_latlong(lat, lon)

    if not state:
        return "Could not determine your location's state. Please try again."

    # Normalize state and crop to match dict keys (Title Case)
    state = state.strip().title()   # "odisha" -> "Odisha"
    crop = crop.strip().title()     # "rice" -> "Rice"
    print(f"Detected State: {state}, Crop: {crop}")

    # Fetch weather (nearest city from state)
    city = state.split()[0]
    temperature, condition = fetch_wttr_weather(city)
    if temperature is None:
        return "Could not fetch weather data."

    # Current month in IST
    ist = pytz.timezone("Asia/Kolkata")
    now = datetime.datetime.now(ist)
    current_month = now.month

    # Check sowing conditions
    can_sow = is_sowing_time(state, crop, current_month, temperature, condition)
    weather_msg = f"Current temperature: {temperature}°C, Condition: {condition}"

    # Get recommended varieties
    variety_info = get_variety_recommendations(state, crop)

    # Final decision
    if can_sow:
        return (
            f"✅ It is a good time to sow {crop} in {state}. {weather_msg}\n\n"
            f"Recommended varieties:\n{variety_info}"
        )
    else:
        return (
            f"❌ Not ideal time to sow {crop} in {state} now. {weather_msg}\n\n"
            f"Recommended varieties for future planning:\n{variety_info}"
        )

In [48]:
crop = "Rice"
result = predict_sowing_time(crop)
print(result)

Detected Location: Latitude=20.2724, Longitude=85.8338
Detected State: Odisha, Crop: Rice
❌ Not ideal time to sow Rice in Odisha now. Current temperature: 25.0°C, Condition: Mist

Recommended varieties for future planning:
- Swarna (Drought tolerant)
- Lalat (High yield)
